<a href="https://colab.research.google.com/github/ahmedmabrouk24/Computer_vision/blob/main/verify_signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json


In [14]:
TRAIN_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train'
TRAIN_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train'
TRAIN_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train'
TRAIN_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train'
TRAIN_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train'
TRAIN_DIR=[]
TRAIN_DIR.append(TRAIN_DIR_A)
TRAIN_DIR.append(TRAIN_DIR_B)
TRAIN_DIR.append(TRAIN_DIR_C)
TRAIN_DIR.append(TRAIN_DIR_D)
TRAIN_DIR.append(TRAIN_DIR_E)


Test_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test'
Test_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test'
Test_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test'
Test_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test'
Test_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test'
Test_DIR=[]
Test_DIR.append(Test_DIR_A)
Test_DIR.append(Test_DIR_B)
Test_DIR.append(Test_DIR_C)
Test_DIR.append(Test_DIR_D)
Test_DIR.append(Test_DIR_E)

PersonA_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train/personA_SigVerificationTrainLabels.csv'
PersonA_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test/personA_SigVerificationTestLabels.csv'

PersonB_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train/personB_SigVerificationTrainLabels.csv'
PersonB_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test/personB_SigVerificationTestLabels.csv'

PersonC_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train/personC_SigVerificationTrainLabels.csv'
PersonC_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test/personC_SigVerificationTestLabels.csv'

PersonD_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train/personD_SigVerificationTrainLabels.csv'
PersonD_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test/personD_SigVerificationTestLabels.csv'

PersonE_train_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train/personE_SigVerificationTrainLabels.csv'
PersonE_test_label='/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test/personE_SigVerificationTestLabels.csv'


train_images=[]
test_images=[]

train_imageA=[]
train_imageB=[]
train_imageC=[]
train_imageD=[]
train_imageE=[]

test_imageA=[]
test_imageB=[]
test_imageC=[]
test_imageD=[]
test_imageE=[]

IMG_SIZE = 200
num_classes = 5
MODEL_NAME = 'signature_model1'


In [15]:
def create_label(image_name,status):
    if image_name[6]=='A' and status=='train':
      df = pd.read_csv(PersonA_train_label)
    elif image_name[6]=='A' and status=='test':
      df = pd.read_csv(PersonA_test_label) 

    elif image_name[6]=='B' and status=='train':
       df = pd.read_csv(PersonB_train_label)
    elif image_name[6]=='B' and status=='test':
       df = pd.read_csv(PersonB_test_label)

    elif image_name[6]=='C' and status=='train':
       df = pd.read_csv(PersonC_train_label)
    elif image_name[6]=='C' and status=='test':
       df = pd.read_csv(PersonC_test_label)

    elif image_name[6]=='D' and status=='train':
       df = pd.read_csv(PersonD_train_label)
    elif image_name[6]=='D' and status=='test':
       df = pd.read_csv(PersonD_test_label)

    elif image_name[6]=='E' and status=='train':
       df = pd.read_csv(PersonE_train_label)
    elif image_name[6]=='E' and status=='test':
       df = pd.read_csv(PersonE_test_label)
    x=False
    for ind in df.index :
      if image_name==df['image_name'][ind]:
        x=True
        if df['label'][ind]=='real':
          return 1
        else:
          return 0
    if x==False:
      print(image_name)


In [ ]:
def create_train_data():
    training_data = []

    for path_TRAIN_DIR in tqdm(TRAIN_DIR):
       for img in os.listdir(path_TRAIN_DIR):
        if 'csv' in img:
          continue
        path = os.path.join(path_TRAIN_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))

        label_list = create_label(img,'train')
        training_data.append([np.array(img_data), label_list])

    random.shuffle(training_data)
    print(len(training_data))
    return training_data

create_train_data()

In [ ]:
column1=[]
def create_test_data():
    test_data = []

    for path_Test_DIR in tqdm(Test_DIR):
       for img in os.listdir(path_Test_DIR):
        if 'csv' in img:
          continue
        column1.append(img)
        path = os.path.join(path_Test_DIR, img)

        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))

        label_list = create_label(img,'test')
        test_data.append([np.array(img_data), label_list])
       

    #random.shuffle(test_data)
    print(len(test_data))
    return test_data
create_test_data()